# CFT Feature Computation

## Setup and Helper Functions

In [ ]:
import re
from pathlib import Path

import pandas as pd
import numpy as np
import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.auto import tqdm

import biopsykit as bp
from biopsykit.protocols import CFT

from cft_analysis.io import load_subject_data_dicts, load_subject_continuous_hrv_data
from cft_analysis.cft import percent_above_baseline_hr, percent_above_baseline_hrv, cft_parameter_per_phase

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
plt.close("all")

palette = bp.colors.fau_palette
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams['figure.figsize'] = (10, 5)
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['mathtext.default'] = "regular"

palette

## Data Import

In [ ]:
data_path = Path("../../../../Data")
ecg_path = data_path.joinpath("ecg")

data_path_export = Path("../../../data/processed")
ecg_path_export = data_path_export.joinpath("ecg")

bp.utils.file_handling.mkdirs([data_path_export, ecg_path_export])

### Load Condition List

In [ ]:
condition_list = bp.io.load_subject_condition_list(data_path.joinpath("condition_list.csv"), return_dict=False)
condition_dict = bp.io.load_subject_condition_list(data_path.joinpath("condition_list.csv"), return_dict=True)
condition_list = condition_list.set_index("condition", append=True)
condition_list.head()

### Load HR(V) Data

In [ ]:
subject_dirs = bp.utils.file_handling.get_subject_dirs(ecg_path, pattern="Vp\w+")

In [ ]:
subject_data_dict_hr, subject_data_dict_rpeaks = load_subject_data_dicts(subject_dirs)

subject_data_dict_hrv = load_subject_continuous_hrv_data(subject_dirs)

In [ ]:
hrv_columns = ["HRV_SDNN", "HRV_RMSSD", "HRV_pNN50", "HRV_pNN20"]

In [ ]:
dict_hr_above_bl = percent_above_baseline_hr(
    subject_data_dict_hr
)
dict_hrv_above_bl = percent_above_baseline_hrv(
    subject_data_dict_hrv,
    hrv_columns
)
dict_cft_params = cft_parameter_per_phase(
    subject_data_dict_hr, 
    condition_dict["CFT"]
)

In [ ]:
level_names = ["subject", "phase", "subphase"]

df_hr_above_bl = bp.utils.data_processing.concat_nested_dict(dict_hr_above_bl, level_names=level_names)
df_hr_above_bl = pd.DataFrame(df_hr_above_bl, columns=["HR"]).droplevel(-1)
df_hr_above_bl = df_hr_above_bl.join(condition_list).reorder_levels(["condition"] + level_names)

df_hrv_above_bl = bp.utils.data_processing.concat_nested_dict(dict_hrv_above_bl, level_names=level_names).unstack()
df_hrv_above_bl = df_hrv_above_bl.join(condition_list).reorder_levels(["condition"] + level_names)

df_hrv_above_bl = df_hrv_above_bl.join(df_hr_above_bl)
df_hrv_above_bl.columns = ["Time_BL_Glo_{}".format(col) for col in df_hrv_above_bl.columns]
df_hrv_above_bl.head()

In [ ]:
level_names = ["subject", "phase"]

df_cft = bp.utils.data_processing.concat_nested_dict(dict_cft_params, level_names=level_names)
df_cft.index = df_cft.index.droplevel(-1)
df_cft = df_cft.join(condition_list).reorder_levels(["condition"] + level_names)

df_cft.head()

In [ ]:
df_hrv_above_bl.to_csv(ecg_path_export.joinpath("cft_hrv_above_baseline.csv"))
df_cft.to_csv(ecg_path_export.joinpath("cft_parameter.csv"))